# Inference with a Pretrained Classifier using Keras NLP

This notebook demonstrates how to use pretrained models for text classification tasks

In [1]:
import tensorflow as tf
import keras_nlp
import numpy as np
import time

print("TensorFlow version:", tf.__version__)
print("Keras NLP version:", keras_nlp.__version__)

TensorFlow version: 2.18.0
Keras NLP version: 0.18.1


In [2]:
# Let's use a pretrained BERT classifier for sentiment analysis
# We'll use the BERT classifier pretrained on IMDB reviews

# Load the pretrained model
model = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en_uncased",     # <-- Corrected: Use the base model preset
    num_classes=2              # Specify number of classes for the head
)

In [4]:

# Let's try inference with some example reviews
positive_review = "This movie was fantastic! I really enjoyed the plot and the acting was superb."
negative_review = "What a waste of time. Poor acting, terrible script, and boring storyline."
mixed_review = "The movie had good special effects but the story was somewhat confusing."

In [5]:

# Let's create a batch of example texts
example_texts = [
    positive_review,
    negative_review,
    mixed_review,
    "I fell asleep halfway through the movie.",
    "The characters were well-developed and the dialogue was engaging.",
    "I can't believe I paid to watch this. It was terrible.",
    "One of the best films I've seen this year!",
    "It was okay, nothing special but not terrible either."
]

In [6]:

# Print review classification with timing
print("\n===== Review Classification =====")
start_time = time.time()

# Get raw predictions
predictions = model.predict(example_texts)
end_time = time.time()

# Convert to predicted class (0 = negative, 1 = positive)
predicted_classes = np.argmax(predictions, axis=1)

# Print results
for i, text in enumerate(example_texts):
    sentiment = "Positive" if predicted_classes[i] == 1 else "Negative"
    confidence = predictions[i][predicted_classes[i]]
    print(f"\nReview: {text}")
    print(f"Prediction: {sentiment} (confidence: {confidence:.4f})")
    print(f"Full logits: {predictions[i]}")

print(f"\nTime taken for {len(example_texts)} predictions: {end_time - start_time:.4f} seconds")


===== Review Classification =====
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step

Review: This movie was fantastic! I really enjoyed the plot and the acting was superb.
Prediction: Negative (confidence: 0.1004)
Full logits: [ 0.10040806 -0.51283854]

Review: What a waste of time. Poor acting, terrible script, and boring storyline.
Prediction: Negative (confidence: 0.1298)
Full logits: [ 0.12977979 -0.4779127 ]

Review: The movie had good special effects but the story was somewhat confusing.
Prediction: Negative (confidence: 0.0717)
Full logits: [ 0.07173215 -0.5124194 ]

Review: I fell asleep halfway through the movie.
Prediction: Negative (confidence: 0.0976)
Full logits: [ 0.09757428 -0.46804893]

Review: The characters were well-developed and the dialogue was engaging.
Prediction: Negative (confidence: 0.0865)
Full logits: [ 0.08649753 -0.49885035]

Review: I can't believe I paid to watch this. It was terrible.
Prediction: Negative (confidence: 0.0791)
Full logits: [ 0.07907256 -0.46644655]



In [7]:

# Let's demonstrate how we can process a single input for real-time applications
print("\n===== Single Input Processing =====")
single_input = "I would recommend this movie to all my friends!"

start_time = time.time()
prediction = model.predict([single_input])
end_time = time.time()

predicted_class = np.argmax(prediction[0])
sentiment = "Positive" if predicted_class == 1 else "Negative"

print(f"Review: {single_input}")
print(f"Prediction: {sentiment} (confidence: {prediction[0][predicted_class]:.4f})")
print(f"Time taken: {end_time - start_time:.4f} seconds")


===== Single Input Processing =====
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Review: I would recommend this movie to all my friends!
Prediction: Negative (confidence: 0.0509)
Time taken: 6.2948 seconds


In [8]:

# Demonstration of how to handle longer text by breaking it into chunks
print("\n===== Processing Longer Text =====")
long_review = """
This movie was a rollercoaster of emotions. The beginning was slow and I almost turned it off,
but I'm glad I didn't because the middle part picked up significantly. The character development
was incredible and by the end I was fully invested in their journey. The cinematography was
breathtaking and the score complemented each scene perfectly. However, some plot points were
left unresolved which was a bit disappointing. Overall, despite its flaws, I would recommend
watching it for the stellar performances of the main cast.
"""

# We could split long text into chunks if needed
chunks = [long_review]  # For this example, we'll process it as one piece

# Process each chunk
chunk_predictions = []
for chunk in chunks:
    prediction = model.predict([chunk])[0]
    chunk_predictions.append(prediction)


===== Processing Longer Text =====
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 580ms/step


In [9]:

# Average predictions across chunks if we had multiple
final_prediction = np.mean(chunk_predictions, axis=0)
final_class = np.argmax(final_prediction)
sentiment = "Positive" if final_class == 1 else "Negative"

print(f"Long Review Analysis:")
print(f"Prediction: {sentiment} (confidence: {final_prediction[final_class]:.4f})")
print(f"Logits: {final_prediction}")

# Let's look at how different models might perform
print("\n===== Comparing Different Pretrained Models =====")
# Note: In a real application, you might want to load these models and compare
# their performance on your specific task

print("Available BERT presets for classification:")
bert_presets = [p for p in keras_nlp.models.BertClassifier.presets if "classifier" in p or "imdb" in p]
for preset in bert_presets:
    print(f" - {preset}")

print("\nNote: You could load different models to compare their performance:")
print("model_1 = keras_nlp.models.BertClassifier.from_preset('bert_base_en_uncased_imdb')")
print("model_2 = keras_nlp.models.RobertaClassifier.from_preset('roberta_base_en_imdb')")

Long Review Analysis:
Prediction: Negative (confidence: 0.1237)
Logits: [ 0.12373428 -0.4878066 ]

===== Comparing Different Pretrained Models =====
Available BERT presets for classification:

Note: You could load different models to compare their performance:
model_1 = keras_nlp.models.BertClassifier.from_preset('bert_base_en_uncased_imdb')
model_2 = keras_nlp.models.RobertaClassifier.from_preset('roberta_base_en_imdb')


In [10]:

# Show how to save and load the model for future use
print("\n===== Saving and Loading Models =====")
# Save to temporary file for demonstration
model_save_path = "/tmp/bert_classifier.keras"
print(f"Saving model to {model_save_path}")
model.save(model_save_path)

print(f"Loading model from {model_save_path}")
loaded_model = tf.keras.models.load_model(model_save_path)

# Verify loaded model works
test_input = "This is a great example of how to use pretrained models."
test_prediction = loaded_model.predict([test_input])
test_class = np.argmax(test_prediction[0])
test_sentiment = "Positive" if test_class == 1 else "Negative"

print(f"Test prediction with loaded model: {test_sentiment}")
print("\nComplete! You've successfully used a pretrained BERT classifier for sentiment analysis.")


===== Saving and Loading Models =====
Saving model to /tmp/bert_classifier.keras
Loading model from /tmp/bert_classifier.keras


/usr/local/lib/python3.11/dist-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Test prediction with loaded model: Negative

Complete! You've successfully used a pretrained BERT classifier for sentiment analysis.
